In [1]:
!pip install -U spacy
!python -m spacy download pt_core_news_md
import spacy


  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: wasabi
    Found existing installation: wasabi 0.8.2
    Uninstalling wasabi-0.8.2:
      Successfully uninstalled wasabi-0.8.2
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\lisat\\anaconda3\\Lib\\site-packages\\~-sly\\msgpack\\_packer.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\lisat\anaconda3\python.exe -m pip install --upgrade pip' command.


  Created wheel for pt-core-news-md: filename=pt_core_news_md-2.3.0-py3-none-any.whl size=50819353 sha256=8c670413b38d45990ab6bdc750f896c9950a6e84de60f9ce28b0d111b365b694
  Stored in directory: c:\users\lisat\appdata\local\pip\cache\wheels\45\7e\5f\9fae426c9fde294fffac05fc382d53bfdcd4a1ad211dd31a20
Successfully built pt-core-news-md
[!] Skipping model package dependencies and setting `--no-deps`. You don't seem
to have the spaCy package itself installed (maybe because you've built from
source?), so installing the model dependencies would cause spaCy to be
downloaded, which probably isn't what you want. If the model package has other
dependencies, you'll have to install them manually.
[+] Download and installation successful
You can now load the model via spacy.load('pt_core_news_md')


You should consider upgrading via the 'C:\Users\lisat\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pt_core_news_md
nlp = pt_core_news_md.load()


In [3]:
def get_np(doc):
        lista_chunks = []
        all_novo_chunk = []
        temNoun=0
        tokens_chunk = []
        for num, token in enumerate(doc):
            if token.pos_ != 'VERB' and token.pos_ != 'AUX' and token.pos_ != 'PUNCT' and token.pos_ != 'PRON' and token.pos_ != 'SPACE' and token.text != '+' and token.text != '#':
                if len(tokens_chunk)!=0 or (len(tokens_chunk)==0 and (token.pos_ != 'ADP' and token.pos_ != 'DET' and token.pos_ != 'SCONJ' and token.pos_ != 'CCONJ')):
                  #chunk = chunk + token.text + ' '
                  tokens_chunk.append(token)
                if token.pos_ == 'NOUN':
                  temNoun=1
            else:
              if len(tokens_chunk)!=0 and temNoun == 1:
                novo_chunk = ''
                # ver se os ultimos tokens não são DET, ADP, etc
                #print(tokens_chunk)
                deve_manter = 0
                for i in range(len(tokens_chunk)-1,-1,-1):
                  #print('i:', i)
                  pos = tokens_chunk[i].pos_
                  texto = tokens_chunk[i].text
                  #print('pos:', pos)
                  #print('texto:', texto)
                  if pos != 'DET' and pos != 'SCONJ' and pos != 'CCONJ' and pos != 'ADP':
                    deve_manter = 1

                  if deve_manter==1:
                    novo_chunk = texto+' '+novo_chunk
                  else:
                    #print('----descartando:-------', texto)
                    pass
                all_novo_chunk.append(novo_chunk.strip())
                # acrescenta na lista
                lista_chunks.append(tokens_chunk)
                tokens_chunk = []
                temNoun = 0
        #print(all_novo_chunk)
        return all_novo_chunk

 
#ex = 'Eu estou tomando um ótimo café em um lugar com excelente vista, com um ótimo café quentinho.'
#ex = 'Há um único quadro de leitura aberto iniciado por códon de metionina de 1.458 nt no quadro com uma homeobox e uma repetição CAX, e o quadro de leitura aberto está previsto para codificar uma proteína de 51.659 daltons.'
ex = 'Data de Criação do Documento: 22/04/2014   Dispneia importante aos esforços + dor tipo peso no peito no esforço. Obeso, has, icc  c # cintilografia miocardica para avaliar angina.'
doc = nlp(ex)

print(get_np(doc))
print(ex)


['Data de Criação do Documento', '22/04/2014 Dispneia importante aos esforços', 'dor tipo peso no peito no esforço', 'Obeso has icc c cintilografia miocardica', 'angina']
Data de Criação do Documento: 22/04/2014   Dispneia importante aos esforços + dor tipo peso no peito no esforço. Obeso, has, icc  c # cintilografia miocardica para avaliar angina.
